In [1]:
import urllib2
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types


from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [2]:
ls

 Volume in drive E has no label.
 Volume Serial Number is 69C0-B1A0

 Directory of E:\Web Data Analytics\Project

06-Dec-17  12:39 PM    <DIR>          .
06-Dec-17  12:39 PM    <DIR>          ..
06-Dec-17  12:38 PM    <DIR>          .ipynb_checkpoints
05-Dec-17  06:57 PM             2,956 .Rhistory
06-Dec-17  12:37 PM             8,300 8.2_MGMT_590_Fall_2017_Google_Cloud_Natural_Language_API.ipynb
04-Dec-17  07:32 PM            46,616 AMEX.csv
05-Dec-17  06:49 PM             5,891 Company List.csv
05-Dec-17  06:55 PM           131,304 company.csv
06-Dec-17  12:15 PM            17,688 Company_list_final.csv
05-Dec-17  07:31 PM            98,080 Dropped companies.csv
06-Dec-17  09:50 AM        34,504,302 earning_calls_data.csv
06-Dec-17  12:39 PM         1,843,936 Earning_Calls_Distributed.ipynb
05-Dec-17  12:07 PM            14,946 final project.ipynb
06-Dec-17  12:14 PM            17,688 John Analysis 2.csv
06-Dec-17  12:13 PM         1,315,867 John Analysis 2.xlsx
06-Dec-17  10:47 AM 

In [10]:
input_earning_calls = pd.read_csv('earning_calls_data.csv', index_col=0)
input_earning_calls = input_earning_calls.drop_duplicates()
input_earning_calls

,Company,Date,Earning_Call
0,IDT,2017-12-05,IDT Corporation (NYSE:IDT)Q1 2018 Earnings Con...
8,WFC,2017-12-05,Wells Fargo & Co. (NYSE:WFC)Goldman Sachs US F...
9,NFLX,2017-12-04,"Netflix, Inc. (NASDAQ:NFLX)UBS 45th Annual Glo..."
10,GM,2017-12-01,General Motors Co. (NYSE:GM)General Motors Co ...
16,CULP,2017-12-01,"Culp, Inc. (NYSE:CULP)Q2 2018 Earnings Confere..."
17,AFL,2017-12-01,Aflac Incorporated (NYSE:AFL)2018 Outlook Tele...
18,F,2017-12-01,Ford Motor Company (NYSE:F)November 2017 U.S. ...
19,RY,2017-11-29,Royal Bank of Canada (NYSE:RY)Q4 2017 Earnings...
20,POT,2017-11-29,PotashCorp Corporation of Saskatchewan Inc. (N...
21,FRO,2017-11-22,Frontline Ltd. (NYSE:FRO)Q3 2017 Earnings Conf...


In [14]:
score = []
magnitude = []
counter = 0
for text in input_earning_calls.Earning_Call.iloc[:10]:
    print(counter)
    counter += 1
    client = language.LanguageServiceClient()
    
    document = types.Document(
    content = text,
    type="PLAIN_TEXT")
    
    sentiment = client.analyze_sentiment(document=document)
    
    score.append(sentiment.document_sentiment.score)
    magnitude.append(sentiment.document_sentiment.magnitude)
    

0
1
2
3
4
5
6
7
8
9


In [15]:
score

[0.0,
 0.10000000149011612,
 0.10000000149011612,
 0.20000000298023224,
 0.10000000149011612,
 0.20000000298023224,
 0.10000000149011612,
 0.10000000149011612,
 0.20000000298023224,
 0.0]

In [16]:
magnitude

[5.900000095367432,
 84.0,
 113.5,
 180.89999389648438,
 69.69999694824219,
 123.4000015258789,
 61.20000076293945,
 77.80000305175781,
 51.5,
 65.30000305175781]